In [1]:
# smallest possible test case for SQL query using jupyter notebook cell magic commands
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

In [2]:
%%sql 
SELECT *
FROM read_json('./data/datathinking.zulipchat.com/raw/messages-000001.json',
               json_format='array_of_records',
               maximum_object_size=99999999,
               columns={
                   'zerver_message': 'STRUCT(id BIGINT, 
                                             sender BIGINT, 
                                             recipient BIGINT, 
                                             realm BIGINT, 
                                             subject VARCHAR, 
                                             content VARCHAR, rendered_content VARCHAR, rendered_content_version BIGINT, 
                                             date_sent TIMESTAMP, sending_client BIGINT, last_edit_time TIMESTAMP, edit_history VARCHAR, has_attachment BOOLEAN, has_image BOOLEAN, 
                                             has_link BOOLEAN, search_tsvector VARCHAR)',
                    'zerver_usermessage': 'STRUCT(id BIGINT, 
                                                  user_profile BIGINT, 
                                                  message BIGINT, 
                                                  flags_mask BIGINT)'
                        },
               lines='true'
);

(duckdb.InvalidInputException) Invalid Input Error: Malformed JSON in file "./data/datathinking.zulipchat.com/raw/messages-000001.json", at byte 2 in line 2: unexpected end of data. 
(Background on this error at: https://sqlalche.me/e/14/f405)


In [3]:
%%sql 
SELECT UNNEST(l) FROM read_json_auto('./data/datathinking.zulipchat.com/raw/messages-000001.json',
               maximum_object_size=99999999) tbl(l);

,unnest(l)
0,"{'id': 299488052, 'sender': 100007, 'recipient..."
1,"{'id': 299488053, 'sender': 100007, 'recipient..."
2,"{'id': 299488054, 'sender': 100007, 'recipient..."
3,"{'id': 327255223, 'sender': 544719, 'recipient..."
4,"{'id': 327934641, 'sender': 588609, 'recipient..."
...,...
235,"{'id': 342181172, 'sender': 595428, 'recipient..."
236,"{'id': 342279369, 'sender': 589761, 'recipient..."
237,"{'id': 342302146, 'sender': 544719, 'recipient..."
238,"{'id': 342314249, 'sender': 544719, 'recipient..."


In [4]:
%%sql 
SELECT UNNEST(zerver_usermessage) AS zerver_usermessage 
FROM read_json_auto('./data/datathinking.zulipchat.com/raw/messages-000001.json',
                    maximum_object_size=99999999);

,zerver_usermessage
0,"{'id': 8257034355, 'user_profile': 544719, 'me..."
1,"{'id': 8257034356, 'user_profile': 544719, 'me..."
2,"{'id': 8257034357, 'user_profile': 544719, 'me..."
3,"{'id': 9332100997, 'user_profile': 587794, 'me..."
4,"{'id': 9332100998, 'user_profile': 544719, 'me..."
...,...
20325,"{'id': 9829132736, 'user_profile': 589802, 'me..."
20326,"{'id': 9829132737, 'user_profile': 590314, 'me..."
20327,"{'id': 9829132738, 'user_profile': 591344, 'me..."
20328,"{'id': 9829132739, 'user_profile': 590323, 'me..."


In [5]:
%%sql 
COPY (SELECT UNNEST(zerver_message) AS zerver_message 
FROM read_json_auto('./data/datathinking.zulipchat.com/raw/messages-000001.json',
                    maximum_object_size=99999999)) TO 'data/datathinking.zulipchat.com/processed/zerver_message.json';

,Count
0,240


In [6]:
%%sql 
COPY (SELECT UNNEST(zerver_usermessage) AS zerver_usermessage 
FROM read_json_auto('./data/datathinking.zulipchat.com/raw/messages-000001.json',
                    maximum_object_size=99999999)) TO 'data/datathinking.zulipchat.com/processed/zerver_usermessage.json';

,Count
0,20330


In [7]:
%%sql 
COPY (SELECT UNNEST(zerver_usermessage) AS zerver_usermessage 
FROM read_json_auto('./data/datathinking.zulipchat.com/raw/messages-000001.json',
                    maximum_object_size=99999999)) TO 'data/datathinking.zulipchat.com/processed/zerver_usermessage.parquet' (FORMAT PARQUET);

,Count
0,20330


In [8]:
%%sql 
SELECT *
FROM 'data/datathinking.zulipchat.com/processed/zerver_usermessage.parquet'

,zerver_usermessage
0,"{'id': 8257034355, 'user_profile': 544719, 'me..."
1,"{'id': 8257034356, 'user_profile': 544719, 'me..."
2,"{'id': 8257034357, 'user_profile': 544719, 'me..."
3,"{'id': 9332100997, 'user_profile': 587794, 'me..."
4,"{'id': 9332100998, 'user_profile': 544719, 'me..."
...,...
20325,"{'id': 9829132736, 'user_profile': 589802, 'me..."
20326,"{'id': 9829132737, 'user_profile': 590314, 'me..."
20327,"{'id': 9829132738, 'user_profile': 591344, 'me..."
20328,"{'id': 9829132739, 'user_profile': 590323, 'me..."


In [9]:
%%sql 
SELECT zerver_usermessage.id, zerver_usermessage.user_profile, zerver_usermessage.message, zerver_usermessage.flags_mask
FROM 'data/datathinking.zulipchat.com/processed/zerver_usermessage.parquet'

,id,user_profile,message,flags_mask
0,8257034355,544719,299488052,1
1,8257034356,544719,299488053,1
2,8257034357,544719,299488054,1
3,9332100997,587794,327255223,1
4,9332100998,544719,327255223,1
...,...,...,...,...
20325,9829132736,589802,342317099,0
20326,9829132737,590314,342317099,0
20327,9829132738,591344,342317099,0
20328,9829132739,590323,342317099,0


In [10]:
%%sql 
SELECT zerver_usermessage.id::UBIGINT as id, 
        zerver_usermessage.user_profile::UBIGINT as user_profile, 
        zerver_usermessage.message::UBIGINT as message, 
        zerver_usermessage.flags_mask::BOOLEAN as flags_mask
FROM 'data/datathinking.zulipchat.com/processed/zerver_usermessage.json'

,id,user_profile,message,flags_mask
0,8257034355,544719,299488052,True
1,8257034356,544719,299488053,True
2,8257034357,544719,299488054,True
3,9332100997,587794,327255223,True
4,9332100998,544719,327255223,True
...,...,...,...,...
20325,9829132736,589802,342317099,False
20326,9829132737,590314,342317099,False
20327,9829132738,591344,342317099,False
20328,9829132739,590323,342317099,False


In [11]:
%%sql 
SELECT zerver_usermessage.id::UBIGINT as id, 
        zerver_usermessage.user_profile::UBIGINT as user_profile, 
        zerver_usermessage.message::UBIGINT as message, 
        zerver_usermessage.flags_mask::BOOLEAN as flags_mask
FROM 'data/datathinking.zulipchat.com/processed/zerver_usermessage.json'

,id,user_profile,message,flags_mask
0,8257034355,544719,299488052,True
1,8257034356,544719,299488053,True
2,8257034357,544719,299488054,True
3,9332100997,587794,327255223,True
4,9332100998,544719,327255223,True
...,...,...,...,...
20325,9829132736,589802,342317099,False
20326,9829132737,590314,342317099,False
20327,9829132738,591344,342317099,False
20328,9829132739,590323,342317099,False


In [12]:
%%sql
SELECT  zerver_message.id::UBIGINT as id,
        zerver_message.sender::UBIGINT as sender,
        zerver_message.recipient::BIGINT as recipient,
        zerver_message.realm::UBIGINT as realm,
        zerver_message.subject::VARCHAR as subject,
        zerver_message.content::VARCHAR as content,
        zerver_message.rendered_content::VARCHAR as rendered_content,
        zerver_message.rendered_content_version::UBIGINT as rendered_content_version,
        zerver_message.date_sent::VARCHAR as date_sent,
        zerver_message.sending_client::BIGINT as sending_client,
        zerver_message.last_edit_time::VARCHAR as last_edit_time,
        zerver_message.edit_history::VARCHAR as edit_history,
        zerver_message.has_attachment::BOOLEAN as has_attachment,
        zerver_message.has_image::BOOLEAN as has_image,
        zerver_message.has_link::BOOLEAN as has_link,
        zerver_message.search_tsvector::VARCHAR as search_tsvector
FROM 'data/datathinking.zulipchat.com/processed/zerver_message.json'

,id,sender,recipient,realm,subject,content,rendered_content,rendered_content_version,date_sent,sending_client,last_edit_time,edit_history,has_attachment,has_image,has_link,search_tsvector
0,299488052,100007,924073,47304,topic demonstration,This is a message on stream #**general** with ...,"<p>This is a message on stream <a class=""strea...",1,1663549367.691131,1002,None,None,False,False,True,"'demonstrate':2,14 'demonstration':2,14 'gener..."
1,299488053,100007,924073,47304,topic demonstration,Topics are a lightweight tool to keep conversa...,<p>Topics are a lightweight tool to keep conve...,1,1663549367.706309,1002,None,None,False,False,True,'conversation':10 'demonstrate':2 'demonstrati...
2,299488054,100007,924073,47304,swimming turtles,This is a message on stream #**general** with ...,"<p>This is a message on stream <a class=""strea...",1,1663549367.719102,1002,None,None,False,True,True,"'general':9 'message':6,28 'new':17 'ply':24 '..."
3,327255223,544719,924073,47304,intros,Hi! I am Jaan. I am teaching this course and t...,<p>Hi! I am Jaan. I am teaching this course an...,1,1676123703.264337,1000,None,None,False,False,False,'agency':32 'artificial':59 'behavior':44 'clo...
4,327934641,588609,924073,47304,intros,"Hello Jaan,\nI am Nesma, a Ph.D. student at th...","<p>Hello Jaan,<br>\nI am Nesma, a Ph.D. studen...",1,1676443984.123736,1163,None,None,False,False,False,'acquire':45 'additional':70 'analysis':58 'ap...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,342181172,595428,924073,47304,plotting,My first plot was about Covid vaccination in d...,<p>My first plot was about Covid vaccination i...,1,1678920353.69432,1000,None,None,True,True,True,'/datasets/uciml/red-wine-quality-cortez-et-al...
236,342279369,589761,924073,47304,GitHub repository,"Hi. Can you (@**Jaan Altosaar** ) pls, explai...","<p>Hi. Can you (<span class=""user-mention"" dat...",1,1678965342.54775,1000,1678965779.853749,"[{""user_id"":589761,""timestamp"":1678965779,""pre...",False,False,False,"'accept':44 'accepted':44 'add':13,51,57 'addr..."
237,342302146,544719,924073,47304,plotting,@_**Gulnara Yusibova|589761** [said](https://d...,"<p><span class=""user-mention silent"" data-user...",1,1678971261.715616,1163,None,None,True,False,True,'/en/copilot/quickstart#getting-your-first-sug...
238,342314249,544719,924073,47304,GitHub repository,@_**Gulnara Yusibova|589761** [said](https://d...,"<p><span class=""user-mention silent"" data-user...",1,1678973924.041869,1163,None,None,False,False,True,"'accept':47 'accepted':47 'add':16,54,60 'addr..."


# Save to parquet

In [13]:
%%sql 
COPY
(SELECT zerver_usermessage.id::UBIGINT as id, 
        zerver_usermessage.user_profile::UBIGINT as user_profile, 
        zerver_usermessage.message::UBIGINT as message, 
        zerver_usermessage.flags_mask::BOOLEAN as flags_mask
FROM 'data/datathinking.zulipchat.com/processed/zerver_usermessage.json'
) TO 'data/datathinking.zulipchat.com/processed/zerver_usermessage.parquet' (FORMAT PARQUET);

,Count
0,20330


In [14]:
%%sql
COPY (
SELECT  zerver_message.id::UBIGINT as id,
        zerver_message.sender::UBIGINT as sender,
        zerver_message.recipient::BIGINT as recipient,
        zerver_message.realm::UBIGINT as realm,
        zerver_message.subject::VARCHAR as subject,
        zerver_message.content::VARCHAR as content,
        zerver_message.rendered_content::VARCHAR as rendered_content,
        zerver_message.rendered_content_version::UBIGINT as rendered_content_version,
        zerver_message.date_sent::VARCHAR as date_sent,
        zerver_message.sending_client::BIGINT as sending_client,
        zerver_message.last_edit_time::VARCHAR as last_edit_time,
        zerver_message.edit_history::VARCHAR as edit_history,
        zerver_message.has_attachment::BOOLEAN as has_attachment,
        zerver_message.has_image::BOOLEAN as has_image,
        zerver_message.has_link::BOOLEAN as has_link,
        zerver_message.search_tsvector::VARCHAR as search_tsvector
FROM 'data/datathinking.zulipchat.com/processed/zerver_message.json'
)
TO 'data/datathinking.zulipchat.com/processed/zerver_message.parquet' (FORMAT PARQUET);

,Count
0,240
